In [1]:
MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings


model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [ ]:
## clean unnessary data

In [3]:

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 

In [5]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(r"C:\Users\DELL\application\WWH\Agriculture\db.pdf")
pages = loader.load_and_split()

In [15]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='llama2', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [6]:
## now, chunks creation
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(pages)

len(all_splits)

9

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(pages, embeddings)

C:\Users\DELL\anaconda3\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
retriever = db.as_retriever()

In [9]:
# prompt
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | model | parser



In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
def chatbot(question):
    # Retrieve context from the vector store
    context = db.similarity_search(question, k=1)[0].page_content

    # Format the prompt with context and question
    prompt_text = prompt.format(context=context, question=question)

    # Get the response from the model
    model_response = model.invoke(prompt_text)
    response = parser.parse(model_response)
    return response


In [12]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = chatbot(user_input)
    print(f"Bot: {response}")

You:  what is pest management for makhana crop?


Bot: Based on the context provided, pest management for Makhana (Euryale ferox) crop involves several techniques to control and prevent pests from damaging the crop. The techniques mentioned in the text are:

1. Remove infected leaves foliage and destroy it safely to prevent the spread of Alternaria leaf blight.
2. Use foliar sprays with copper oxychloride or dithane M-45 @ 0.3% twice or thrice at fortnightly interval to check the disease.
3. For root rot disease, there is no known cure, so it is vital to remove and destroy any infected plants.

It's important to note that the management of pests in Makhana crop may vary depending on the severity of the infestation and the specific location where it is grown. Therefore, it is recommended to consult with local agricultural experts or researchers for the most effective and appropriate pest management techniques for the specific region.


You:  exit


In [13]:
response

"Based on the context provided, pest management for Makhana (Euryale ferox) crop involves several techniques to control and prevent pests from damaging the crop. The techniques mentioned in the text are:\n\n1. Remove infected leaves foliage and destroy it safely to prevent the spread of Alternaria leaf blight.\n2. Use foliar sprays with copper oxychloride or dithane M-45 @ 0.3% twice or thrice at fortnightly interval to check the disease.\n3. For root rot disease, there is no known cure, so it is vital to remove and destroy any infected plants.\n\nIt's important to note that the management of pests in Makhana crop may vary depending on the severity of the infestation and the specific location where it is grown. Therefore, it is recommended to consult with local agricultural experts or researchers for the most effective and appropriate pest management techniques for the specific region."